### Sequence data

* 한 단어만으로는 전체 맥락을 이해하지 못한다.
* 앞의 단어와 현재의 단어를 기반으로 이해해야 한다.(the previous word + this word, time series)
* NN/CNN은 이러한 sequence data를 구현하기 어렵다.

* 즉, 현재의 상태가 다음 상태에 영향을 줄 수 있는 네트워크 모델이 필요하다.

### Recurrent Neural Network
* 현재의 상태를 계산하고, 다시 그것을 입력으로 넣는 형태의 모델
* 각 RNN에서 Y_hat(예측값)을 뽑아낼 수 있다. 계산방법은 다음과 같다.

$$h(현재상태)= f_w(h_(이전상태), x) $$

* fw는 weight를 갖는 함수로 모든 RNN에서 동일한 함수를 사용한다.

* 입력이 두개이므로 weight도 두개가 필요하다. 예를 들어(Vanilla RNN)

$$ h(현재상태) = tanh(W_1 h(이전상태) + W_2 X)$$

$$ y(현재 상태) = W_3 h(현재상태)$$

* h 는 일반적으로 hidden layer라고 하고, h(이전상태)는 초기에는 없기 때문에 일반적으로 0으로 설정

### RNN의 여러가지 형태
* one to one : Varinilla RNN
* one to many : Image Captioning
* many to one : sentiment classification. sequence of words -> sentiment
* many to many : 기계 번역. seq of words -> seq of words
* many to many : 프레임 차원의 동영상 분류 -> 설명 출력

$$$$
* Multi-layer RNN 도 가능하며, 마찬가지로 layer가 깊어지면 학습이 어려워진다. 이를 개선한 모델이 LSTM(Long Short Term Memory)이다.

### RNN in TensorFlow

In [2]:
import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.ERROR)
tf.set_random_seed(777)

### One node: 4 (input-dim) in 2 (hidden_size)

* hidden_size를 어떻게 설정할 지에 따라 output의 dimension이 달라진다. 여기에서는 2로 설정한다.


* input shape = (1, 1, 4)  [[1, 0, 0, 0]]

In [3]:
# 이 사례에서는 training이 없음
hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
#cell = tf.keras.layers.LSTMCell(units=hidden_size)

x_data = np.array([[[1, 0, 0, 0]]], dtype=np.float32) # 'h'
#quesiton = tf.keras.layers.Input(shape=(1, 1, 4), dtype=tf.float32)

outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
#outputs = tf.keras.layers.RNN(cell, dtype=tf.float32)(x_data)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

import pprint as pp
pp.pprint(outputs.eval(session=sess))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

array([[[0.04945214, 0.04316235]]], dtype=float32)


hidden_size를 2로 설정하였기 때문에 outputs 값도 두개가 나왔다.

### sequence input data의 경우  : hidden_size = 2, sequence_length = 5 (hello)

* 이 경우, input data의 shape은 (1, 5, 4). 이것을 각 문자별로 펼쳐놓는다 (unfolding to n sequences)

  hidden_size = 2
  
  sequence_length = 5 ( 입력 단어 수 )

 이렇게 되면 출력 shape = (1, 5, 2) 가 된다.

In [8]:
tf.reset_default_graph()
tf.set_random_seed(777)
# one hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
x_data = np.array([[h, e, l, l, o]], dtype=np.float32)  # sequence data
print(x_data.shape)
pp.pprint(x_data)

outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)


In [10]:
# outputs
sess = tf.Session()
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval(session=sess))
print(outputs)

array([[[ 0.04945214,  0.04316235],
        [-0.00104636,  0.06992362],
        [ 0.06311742, -0.06535073],
        [ 0.11887082, -0.16511463],
        [ 0.16751593, -0.26476774]]], dtype=float32)
Tensor("rnn/transpose_1:0", shape=(1, 5, 2), dtype=float32)


### Batching input

* sequence를 하나씩 주는 것이 아니라 여러개를 줄 경우, 이를 batch_size로 표현 가능하다.

* 예를 들어 hello, eolll, lleel shape = (3, 5, 4) 이면 batch_size = 3이 된다.
* 이 경우 outputs의 shape = (3, 5, 2) 가 된다.

In [13]:
tf.reset_default_graph()
hidden_size = 2
sequece_length = 5
batch_size = 3
x_data = np.array([[h, e, l, l, o], 
                   [e, o, l, l, l], 
                   [l, l, e, e, l]], dtype=np.float32)

pp.pprint(x_data)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval(session=sess))
print(outputs)

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.04283834, -0.05121393],
        [-0.03094209,  0.10285935],
        [ 0.11129723, -0.03200451],
        [ 0.22397222, -0.12367146],
        [ 0.21238048, -0.0769559 ]],

       [[ 0.01226371,  0.13834858],
        [ 0.08454693,  0.03562944],
        [ 0.21645911, -0.06988609],
        [ 0.29929954, -0.14282013],
        [ 0.35645092, -0.18373272]],

       [[ 0.13940895, -0.10760844],
        [ 0.24243172, -0.16845688],
        [ 0.16465393,  0.04037705],
        [ 0.16442199,  0.17125313],
        [ 0.30980796,  0.02960309]]], dtype=float32)
Tensor("rnn/transpose_1:0", 